# Orbit response

In [ ]:
import sys
import os
from os.path import join
from collections import Counter
from datetime import datetime
import h5py
from tqdm import tqdm
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt
import proplot as pplt

sys.path.append('/Users/46h/Research/code/optimized_sav_gal/')
import sgfilter

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'

In [ ]:
def get_quad_id(filename):
    return filename.split('_')[5]

def get_quad_number(filename):
    return int(get_quad_id(filename)[2:])

def mean_std(x, f):
    N = np.sum(f)
    mean = np.sum(f * x) / N
    std = np.sum(f * (x - mean)**2) / N
    return mean, std

def autofilter(signal):
    n_opt = sgfilter.n_opt(signal)
    signal_filtered = sgfilter.sg_filter_gram(signal, n_opt, 2)
    return signal_filtered.values

In [ ]:
folder = 'Diagnostics/Data/Measurements/2022-04-22/'
filenames = os.listdir(folder)
filenames = [filename for filename in filenames if 'orbit_response' in filename]
filenames = sorted(filenames, key=get_quad_number, reverse=True)
filenames

In [ ]:
filename = filenames[0]
file = h5py.File(join(folder, filename), 'r')
list(file.keys())

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['scandata']
print('Data:')
for i, name in enumerate(data.dtype.names):
    print(f'{name} [{data.dtype[i]}]')
print()

slits = [key for key in data.dtype.names if 'PositionSync' in key]
print(f'slits: {slits}')

In [ ]:
for name in data.dtype.names:
    if name in ['timestamp', 'Cam09_profileX', 'Cam09_profileY']:
        continue
    fig, ax = pplt.subplots(figsize=(5.0, 2.0))
    ax.plot(data['iteration'], data[name], color='black')
    for i in [22, 44, 66, 88, 110]:
        ax.axvline(i, color='black', alpha=0.1)
    ax.format(xlabel='Iteration', ylabel=name)
    plt.show()

Some of the signals are just noise.

In [ ]:
profile = data['Cam09_profileX'][100]
profile = profile[75:]  # No signal at beginning.
noise_mean = np.mean(profile)

fig, ax = pplt.subplots(figsize=(4.0, 1.5))
ax.plot(profile, color='black', alpha=0.1, label='Signal')
ax.axhline(noise_mean, color='black', label='Noise mean')
ax.legend(loc='top')
plt.show()

Try thresholding to get rid of noise.

In [ ]:
thresh = 2e5
good_idx = []
good_profiles = []
good_profiles_filtered = []
for i, profile in enumerate(tqdm(data['Cam09_profileX'])):
    profile = profile[75:]
    profile -= noise_mean
    profile[profile < thresh] = thresh
    profile -= thresh
    if np.count_nonzero(profile) > 60:
        good_idx.append(i)
        good_profiles.append(profile)
        good_profiles_filtered.append(autofilter(profile))
good_idx = np.array(good_idx)

In [ ]:
n = 10
means = []
for i, f, ff in zip(good_idx, good_profiles, good_profiles_filtered):
    imax = np.argmax(ff)
    means.append(imax)
#     fig, ax = pplt.subplots(figsize=(4, 1.5))
#     ax.plot(f, color='black', alpha=0.1)
#     ax.plot(ff, color='black')
#     ax.axvline(imax, color='pink7', alpha=0.5)
#     plt.show()

In [ ]:
means_long = np.full(len(data), np.nan)
for i, mean in zip(good_idx, means):
    means_long[i] = mean

fig, ax = pplt.subplots(figsize=(9, 2))
ax.plot(means_long, color='black', lw=None, marker='.', ms=3)
ax.format(ylabel='Mean position', xlabel='Step')
plt.show()

One iteration = one sweep of the first slit. One iteration should be equal to 22 steps. I will let one "period" = one sweep of the second slit. One period should be equal to 22 iterations. The quad current is changed after each period.

Let's get the step indices for each period.

In [ ]:
quad_id = get_quad_id(filename)
period_length = 22
idx_dict = dict()
for period in range(1, 7):
    idx = np.logical_and(
        data['iteration'] >= (period - 1) * period_length + 1,
        data['iteration'] <= period * period_length,
    )
    idx_dict[period] = data['iteration'][idx]
idx_dict

Recall that our goal is to compute the beam centroid for each x,x' as a function of quad current. First, let's get the data from this period.

In [ ]:
period = 1
idx = idx_dict[period]
slit_pos = {slit: data[slit] for slit in slits}
quad_current_set = np.mean(data[f'{quad_id}_CurrentSet'][idx])
quad_current_avg = np.mean(data[f'{quad_id}_CurrentRb'][idx])
xprofiles = data['Cam09_ProfileX'][idx, :]
yprofiles = data['Cam09_ProfileY'][idx, :]
print(f'quad_current_avg = {quad_current_avg:.2f}')
print(f'quad_current_aset = {quad_current_avg:.2f}')

For some reason, there are more than 22 steps (positions of the sweeping slit) per iteration. Furthermore, the number of steps per iteration changes on each iteration. I'm not sure why.

In [ ]:
steps_per_iter = Counter(idx)
print('Iteration: number of steps')
steps_per_iter

We could cut them off at the same number; hold off for now.

Let's look at the profiles at the first iteration — as the first slit sweeps.

In [ ]:
iteration = 1
steps = steps_per_iter[iteration]
lo = iteration - 1
hi = lo + steps
_yprofiles = yprofiles[lo:hi, :]

for i, y in enumerate(_yprofiles):
    x = 0.023 * np.arange(len(y))  # 0.024 mm/pixel at zoom=1.0.
    y_filtered = autofilter(y)
    imean = np.argmax(y_filtered)
    mean = x[imean]
    print(f'mean = {mean:.2f}')
#     fig, ax = pplt.subplots()
#     ax.plot(x, y, color='black', alpha=0.1, label='original')
#     ax.plot(x, y_filtered, color='black', label='filtered')
#     ax.format(yticklabels=[], ylabel='Amplitude', xlabel='y [mm]')
#     ax.axvline(mean, color='pink8', alpha=0.5, label='mean')
#     ax.legend(loc='top', framealpha=0.0)
# #     plt.savefig(f'_output/Cam09_ProfileY_{i}')
#     plt.show()

I want to know: for a given x and x', how does it's position on the screen change as a function of quad strength? That means that I need to match up iterations between periods.

In [ ]:
i = 4
for j in range(12):
    idx = np.logical_and(
        data[slits[1]] == data[slits[1]][i],
        data[slits[0]] == data[slits[0]][j],
    )

    y = data['Cam09_ProfileY'][idx, :]
    if y.shape[0] > 1:
        y = y[0]
    y = np.squeeze(y)
    x = 0.023 * np.arange(len(y))  # 0.024 mm/pixel at zoom=1.0.
    
    # Check noise level.
    peak = np.max(y) / np.mean(y)
    print(peak)

    fig, ax = pplt.subplots()
    ax.plot(x, y, color='black', alpha=0.1)
    
#     y_filtered = autofilter(y)
#     ax.plot(x, y_filtered, color='black')
#     imean = np.argmax(y_filtered)
#     mean = x[imean]
#     ax.format(yticklabels=[], ylabel='Amplitude', xlabel='y [mm]')
#     ax.axvline(mean, color='pink8', alpha=0.5, label='mean')
#     plt.show()